# 4.1.Sentiment Analysis | Messages
This part of sentiment analysis consists of finding the polarity of the text messages. 
Due to dependence on the libraries of the virtual environment. For text sentiment analysis I have to use a different environment than the one previously used (sentiment_analysis).

In [1]:
import pandas as pd
import re

In [2]:
data = pd.read_csv('data.csv')

In [3]:
data.shape

(9770, 13)

## Clean msg | Apply sentiment analysis to clean msg

In [4]:
import ssl
from datetime import timedelta
from datetime import datetime
import timeit
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load pre-trained model and tokenizer
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model.eval()

def get_sentiment_token(text):
    # Tokenize the text
    tokens = tokenizer.encode(text, return_tensors='pt', truncation=True, max_length=512)
    
    # Forward pass through the model
    with torch.no_grad():
        outputs = model(tokens)
    
    # Get the sentiment polarity (index of the maximum logit)
    sentiment = torch.argmax(outputs.logits, dim=1).item() + 1
    
    return sentiment

# Example usage
text = "I love this product! It works great and I couldn't be happier."
sentiment = get_sentiment_token(text)
print(f"Sentiment score: {sentiment}")

Sentiment score: 5


In [5]:
import torch.nn.functional as F

# Load pre-trained model and tokenizer
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

def get_sentiment_token(text):
    # Tokenize the text
    tokens = tokenizer.encode(text, return_tensors='pt', truncation=True)
    
    # Check if the tokens exceed the maximum sequence length
    if tokens.size(1) > 512:
        print(f"Tokens size exceeds maximum sequence length: {tokens.size(1)}")
        # Pad the tensor if needed
        tokens = F.pad(tokens, (0, 512 - tokens.size(1)))
    
    # Forward pass through the model
    blob = model(tokens)
    
    # Return the sentiment polarity
    return int(torch.argmax(blob.logits)) + 1

In [6]:
import timeit
start_time = timeit.default_timer()

get_sentiment_token('no me gusta eso')

end_time = timeit.default_timer()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time} seconds")

Elapsed time: 0.04257489999872632 seconds


In [7]:
import timeit
start_time = timeit.default_timer()

# Create a copy of the DataFrame to avoid SettingWithCopyWarning
# data_copy = data.copy()

# Apply sentiment analysis to each row of the 'clean_msg' column
data['Sentiment_Polarity'] = data['clean_msg'].apply(lambda x: get_sentiment_token(x))

# Map sentiment polarity to labels
data['Sentiment_Label'] = data['Sentiment_Polarity'].map({5: 'Positive', 1: 'Negative', 3: 'Neutral'})

end_time = timeit.default_timer()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time} seconds")

Elapsed time: 661.270815099997 seconds


In [8]:
# Save the labeled data to a CSV file
data.to_csv('data_labeled.csv', index=False)